# FOR TRAINING DATASET

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import figure
import mne
import os, shutil, fnmatch
import random
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

In [1]:
def find_min_num_of_file(dir):

    print(f'======== The process of finding the optimal number of images for each stage has been started. ========')
    print(f'======== During the process files of the supplied directory "dir" will be analysed. ========\n')

    all_files_in_dir = os.listdir(dir)
    stages = 6*[0]

    all_psg_files = []
    all_hyp_files = []
    checked = 0
    annotation_desc_2_event_id = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    for file in all_files_in_dir:
        if file[0] == '.': # for ignoring hidden files in the directory
            continue
        parts = file.split("-")
        if parts[1] == "PSG.edf":
            all_psg_files.append(file)
        elif parts[1] == "Hypnogram.edf":
            all_hyp_files.append(file)

    for file in all_psg_files:

        current_psg_file = dir + "/" + file

        hyp_file = file.split("-")[0][:-2] + "*" + "-Hypnogram.edf"
        possible_hyp_file = fnmatch.filter(all_hyp_files, hyp_file)

        if possible_hyp_file:

            current_hyp_file = dir + "/" + possible_hyp_file[0]

            data = mne.io.read_raw_edf(current_psg_file, stim_channel="Event marker", infer_types=True, preload=True, verbose=False)

            annot_train = mne.read_annotations(current_hyp_file)

            annot_train.crop(annot_train[1]["onset"] - 30*60, annot_train[-2]["onset"] + 30*60)
            data.set_annotations(annot_train, emit_warning=False)

            events_from_the_file, _ = mne.events_from_annotations(
                data, event_id=annotation_desc_2_event_id, chunk_duration=30.0, verbose=False
            )

            for event in events_from_the_file:
                stages[list(annotation_desc_2_event_id.values()).index(event[2])] += 1

            checked += 1
            print(f"======== Files have been analysed: {checked}/{len(all_psg_files)} ========", end="\r")
        else:
            print(f"\n======== No such hypnogram file for {file} ========\n")

    print("\n======== Analysing is ended. ========\n")
    i = 0
    for stage in list(annotation_desc_2_event_id):
        print(f"======== Number of files for {stage} is {stages[i]} ========")
        i += 1

    print(f"======== Minimum number of the files for each stage is {min(stages)} ========")

    return min(stages), max(stages)

In [2]:
def file_keeper(psg_file_path, hyp_file_path):

    data = mne.io.read_raw_edf(psg_file_path, stim_channel="Event marker", infer_types=True, preload=True)
    annotations = mne.read_annotations(hyp_file_path)

    data.filter(0.5, 30, picks=[0])

    return data, annotations

In [3]:
def crop_set_annotations(data, annotations):

    annotations.crop(annotations[1]["onset"] - 30*60, annotations[-2]["onset"] + 30*60)
    data.set_annotations(annotations, emit_warning=False)

    annotations_stage_id = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    events_from_the_file, event_id_info = mne.events_from_annotations(
        data, event_id= annotations_stage_id, chunk_duration= 30.0
    )

    return events_from_the_file, event_id_info, annotations_stage_id

In [4]:
def create_directories():

    directories = ()
    dir_name = "dataset"

    sleep_stages = {
        "Sleep stage W": 1,
        "Sleep stage 1": 2,
        "Sleep stage 2": 3,
        "Sleep stage 3": 4,
        "Sleep stage 4": 5,
        "Sleep stage R": 6,
    }

    if not os.path.isdir(dir_name):

        main_dir = dir_name
        train_dir = dir_name + "/train"
        test_dir = dir_name + "/test"
        valid_dir = dir_name + "/valid"

        train_files_dir = []
        test_files_dir = []
        valid_files_dir = []

        for i in sleep_stages:
            if i == 'Sleep stage ?':
                pass
            else:
                train_files_dir.append(dir_name + "/train/" + i)
                test_files_dir.append(dir_name + "/test/" + i)
                valid_files_dir.append(dir_name + "/valid/" + i)
    
        directories = (main_dir, train_dir, train_files_dir, test_dir, test_files_dir, valid_dir, valid_files_dir )
    
        for directory in directories:
            if isinstance(directory, list):
                for sub_directory in directory:
                    if os.path.isdir(sub_directory):
                        pass
                    else:
                        os.mkdir(sub_directory)
    
            else:
                if os.path.isdir(directory):
                    pass
                else:
                    os.mkdir(directory)
    
        print("======== Directories have been created. ========")
    else:
        print("Dataset directory already exists.")
        raise SystemExit("Please, remove old dataset directory!")
                

In [5]:
def create_images(data, events_from_the_file, event_id_info, annotations_stage_id, psg_file, optim_num_imgs, ctrl_imgs, place_for_image):

    tmax = 30.0 - 1.0 / data.info["sfreq"]
    print(f"Number of epochs in the events_from_the_file for the file: {len(events_from_the_file)}, number of images to be created: {optim_num_imgs}")

    epochs_from_the_file = mne.Epochs(
        raw=data,
        events=events_from_the_file,
        event_id=event_id_info,
        tmin=0.0,
        tmax=tmax,
        baseline=None,
        verbose=False
    )
    events = epochs_from_the_file.get_data(picks=[0])
    times_events = epochs_from_the_file.events

    for iteration in range(len(events)):
        
        start_time = int(times_events[iteration][0] / data.info.get('sfreq'))   # start moment of the signal part on the graph

        if ctrl_imgs[times_events[iteration][2] - 1] < optim_num_imgs:

            plt.ioff()

            fig = figure.Figure(figsize=(8,8))
            ax = fig.add_subplot(111)

            ax.plot(events[iteration][0], linewidth=0.5, color="black")

            ax.axis("off")

            stage_list = list(annotations_stage_id.keys())
            stage_id = list(annotations_stage_id.values()).index(times_events[iteration][2])
            path_for_image = f"dataset/{place_for_image}/" + stage_list[stage_id] + "/" + stage_list[stage_id] + \
                    '_' + psg_file.split("-")[0] + '_' + str(start_time)

            fig.savefig(path_for_image, bbox_inches='tight')

            plt.close(fig)

            ctrl_imgs[times_events[iteration][2] - 1] += 1

    print(f"\nImages have been prepared for {psg_file.split('-')[0][0:-2]}.\n")

In [6]:
def dataset_create(files_directory, optim_num_imgs, place_for_image):

    all_files = os.listdir(files_directory)
    not_processed_files = []
    all_psg_files = []
    all_hyp_files = []

    ctrl_imgs = 6*[0] # List for the number of images (for 6 stages)

    for file in all_files:

        if file[0] == '.': # for ignoring hidden files in the directory
            continue

        parts = file.split("-")

        if parts[1] == "PSG.edf":
            all_psg_files.append(file)
        elif parts[1] == "Hypnogram.edf":
            all_hyp_files.append(file)

    # for controlling status progress
    old_process_percentage = 0
    process_percentage = 0
    iteration = 0

    for psg_file in all_psg_files:

        hyp_file = psg_file.split("-")[0][:-2] + "*" + "-Hypnogram.edf"
        possible_hyp = fnmatch.filter(all_hyp_files, hyp_file)

        if possible_hyp:

            hyp_file = possible_hyp[0]

            print(f"\n================ Files currently being processed: {psg_file}, {hyp_file} ================")

            psg_file_path = files_directory + "/" + psg_file
            hyp_file_path = files_directory + "/" + hyp_file

            data, annotations = file_keeper(psg_file_path, hyp_file_path)
            print("======== Got data and annotations. ========")

            events_from_the_file, event_id_info, annotations_stage_id = crop_set_annotations(data, annotations)
            print("======== Annotations cropped and set. ========")

            create_images(data, events_from_the_file, event_id_info, annotations_stage_id, psg_file, optim_num_imgs, ctrl_imgs, place_for_image)

        else:
            not_processed_file = psg_file.split("-")[0][:-2] # Get number of the candidate, i.e. SC4812
            print(f"No such hypnogram file for {not_processed_file}")
            not_processed_files.append(not_processed_file)

        iteration += 1

        process_percentage = round(iteration / len(all_psg_files) * 100)  # progress controlling

        if process_percentage != old_process_percentage:
            print(f"======== Extracting images from PSG signals: {process_percentage}% ========", end="\r")

        old_process_percentage = process_percentage

    print("\nEND. Images for the dataset have been created.")
    if not_processed_files:
        print(f"Files that weren't processed: {not_processed_files}")

## FOR PREDICTING DATASET

In [7]:
def create_dir_and_img_pred(name_predict_sig, data, events_from_the_file, event_id_info, annotations_stage_id, psg_file):

    # Creating a directory

    dir_name = "dataset_predict"

    name_predict_sig = dir_name + "/" + name_predict_sig    # creating  a full path of the folder

    directories = (dir_name, name_predict_sig)

    for directory in directories:
        if os.path.isdir(directory):
            pass
        else:
            os.mkdir(directory)
            print(f"======== Directory {directory} has been created. ========")

    # Creating an image

    tmax = 30.0 - 1.0 / data.info["sfreq"]
    
    epochs_from_the_file = mne.Epochs(
        raw=data,
        events=events_from_the_file,
        event_id=event_id_info,
        tmin=0.0,
        tmax=tmax,
        baseline=None,
        verbose=False
    )
    events = epochs_from_the_file.get_data(picks=[0])
    times_events = epochs_from_the_file.events

    i = 0
    old_process_percntage = 0
    process_percentage = 0

    for iteration in range(len(events)):
        
        process_percentage = round(i / len(events) * 100)    # process controlling
        if process_percentage != old_process_percntage:
            print(f"======== Extracting images from PSG signals: {process_percentage}% ========", end="\r")

        start_time = int(times_events[iteration][0] / data.info.get('sfreq'))   # start moment of the signal part on the graph

        plt.ioff()

        fig = figure.Figure(figsize=(8,8))
        ax = fig.add_subplot(111)

        ax.plot(events[iteration][0], linewidth=0.5, color="black")

        ax.axis("off")

        stage_list = list(annotations_stage_id.keys())
        stage_id = list(annotations_stage_id.values()).index(times_events[iteration][2])

        path_for_image = name_predict_sig + '/' + stage_list[stage_id] + \
                        '_' + psg_file.split("-")[0] + '_' + str(start_time)

        fig.savefig(path_for_image, bbox_inches='tight')

        plt.close(fig)

        old_process_percentage = process_percentage

        i += 1

    print(f"\n Images have been preapred for {psg_file.split('-')[0][0:-2]}.\n")     

In [8]:
def create_predict_dataset(files_directory):

    print(f"======== The process of predict dataset has been started. ========")
    print(f"======== During the process files of the supplied directory will be processed. ========\n")

    all_files = os.listdir(files_directory)
    not_processed_files = []
    all_psg_files = []
    all_hyp_files = []

    for file in all_files:
        if file[0] == '.': # for ignoring hidden files
            continue
        parts = file.split("-")
        if parts[1] == "PSG.edf":
            all_psg_files.append(file)
        elif parts[1] == "Hypnogram.edf":
            all_hyp_files.append(file)

    old_process_percentage = 0
    process_percentage = 0

    iteration = 0

    for psg_file in all_psg_files:

        process_percentage = round(iteration / len(all_psg_files) * 100) # process controlling
        if process_percentage != old_process_percentage:
            print(f"\n======== Extracting images from PSG signals: {process_percentage}% ========\n")

        hyp_file = psg_file.split("-")[0][:-2] + "*" + "-Hypnogram.edf"
        possible_hyp = fnmatch.filter(all_hyp_files, hyp_file)

        if possible_hyp:
            hyp_file = possible_hyp[0]

            print(f"======== Files currently being processed: {psg_file}, {hyp_file} ========")

            psg_file_path = files_directory + "/" + psg_file
            hyp_file_path = files_directory + "/" + hyp_file

            data, annotations = file_keeper(psg_file_path, hyp_file_path)
            print("======== Got data and annotations. ========")

            events_from_the_file, event_id_info, annotations_stage_id = crop_set_annotations(data, annotations)
            print("======== Annotations cropped and set. ========")

            name_predict_sig = psg_file.split("-")[0][:-2]
            create_dir_and_img_pred(name_predict_sig, data, events_from_the_file, event_id_info, annotations_stage_id, psg_file)

        else:
            not_processed_file = psg_file.split("-")[0][:-2] # Gets number of the candidate, i.e. SC4812
            print(f"No such hypnogram file for {not_processed_file}")
            not_processed_files.append(not_processed_file)

        old_process_percentage = process_percentage

        iteration += 1

    print(f"\n======== Extracting images from PSG signals: {process_percentage}% ========\n")


    print("\n======== END. Images for the dataset have been created. ========")

    if not_processed_files:
        print(f"Files that weren't processed: {not_processed_files}")

# FOR MODEL TRAINING

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [9]:
def preprocessing_func():

    train_transform = ImageDataGenerator(rescale=1./255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True)
    training_set = train_transform.flow_from_directory('dataset/train',
                                                      target_size=(800,800),
                                                      color_mode="grayscale",
                                                      batch_size=32,
                                                      class_mode='categorical')

    
    validation_transform = ImageDataGenerator(rescale=1./255)
    validation_set = validation_transform.flow_from_directory('dataset/valid',
                                                      target_size=(800,800),
                                                      color_mode="grayscale",
                                                      batch_size=32,
                                                      class_mode='categorical')


    test_transform = ImageDataGenerator(rescale=1./255)
    test_set = test_transform.flow_from_directory('dataset/test',
                                                      target_size=(800,800),
                                                      color_mode="grayscale",
                                                      batch_size=32,
                                                      class_mode='categorical')

    return training_set, validation_set, test_set

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_cnn():

    cnn = Sequential()

    cnn.add(Conv2D(16, (3,3), activation='relu', input_shape=[800, 800, 1]))
    cnn.add(MaxPooling2D((2,2)))
    
    cnn.add(Conv2D(32, (3,3), activation='relu'))
    cnn.add(MaxPooling2D((2,2)))

    cnn.add(Conv2D(64, (3,3), activation='relu'))
    cnn.add(MaxPooling2D((2,2)))

    cnn.add(Flatten())

    cnn.add(Dense(64, activation='relu'))
    cnn.add(Dense(units=6, activation='softmax'))

    return cnn

In [ ]:
def cnn_train_model(cnn, training_set, validation_set):

    cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    cnn.fit(x=training_set, validation_data=validation_set, epochs=5)
    if os.path.isdir('models'):
        print(f"Folder 'models' already exists.")
        cnn.save('models/exit_model.keras')
    else:
        os.mkdir('models')
        cnn.save('models/exit_model.keras')

In [ ]:
def check_for_hid_files():   # for removing "." files

    print("======== Checking on hidden '.' files started. ========")

    datasets_for_check = ["dataset", "dataset_predict"]

    for dataset in datasets_for_check:
        if not os.path.isdir(dataset):
            print(f"Directory '{dataset}' doesn't exist.")
            continue

        for folder in os.listdir(dataset):
            dir_1 = dataset + "/" + folder
            if folder[0] == ".":
                os.remove(dir_1)

            for folder_el in os.listdir(dir_1):
                dir_2 = dir_1 + "/" + folder_el
                if folder_el[0] == ".":
                    os.remove(dir_2)
                if dataset == "dataset":
                    for element in os.listdir(dir_2):
                        if element[0] == ".":
                            dir_rem = dir_2 + "/" + element
                            os.remove(dir_rem)

        if os.path.isdir(dataset):
            print(f"======== Dataset '{dataset}' has been checked for hidden files. ========")

# Predicting

In [ ]:
def create_hypnograms(events, times_events, sfreq, img_folder_path, cnn, annotation_stage_id, training_set):

    # Coordinates for true hypnogram and predicted respectively
    x = []
    y = []

    x_pred = []
    y_pred = []

    percent = 0
    correct_predict = 0
    true_list = 6*[0] # List for true numbers of the each stage, based on the annotation_stage_id
    correct_predict_list = 6*[0] # List of the correct predictions for the each stage, based on the annotation_stage_id

    stages_names_list = list(annotation_stage_id.keys())
    stages_id_list = list(annotation_stage_id.values())

    for iteration in range(1, len(times_events)):

        start = int(times_events[iteration - 1][0] / sfreq)
        duration = int(times_events[iteration][0] / sfreq) - start
        stage_id = times_events[iteration - 1][2]

        current_stage_name = stages_names_list[stages_id_list.index(stage_id)]

        x.append(start)
        y.append(current_stage_name)
        x.append(start + duration)
        y.append(current_stage_name)

        image_name = "*" + str(start) + ".png"  # image name of the signal for specific PSG folder in the dataset_predict
        possible_image = fnmatch.filter(os.listdir(img_folder_path), image_name)[0]

        if not possible_image:
            print(f"Possible image file {image_name} hasn't been found.")
            continue

        true_list[stage_id - 1] += 1  # counting stage elements

        # print(f"Image that is currently being processed: {possible_image}, start time: {start}")

        path_possible_img = img_folder_path + "/" + possible_image

        expan_dim_image = image.load_img(path_possible_img, color_mode='grayscale', target_size=(800, 800))
        expan_dim_image = image.img_to_array(expan_dim_image)
        expan_dim_image = np.expand_dims(expan_dim_image, axis=0)
        predict_res = cnn.predict(expan_dim_image)
        
        # print(f"Predicted raw result: {predict_res}")
        # print(f"Class indexes: {training_set.class_indices}")

        index_predict = list(predict_res[0])
        # print(index_predict)

        
        predicted_value = np.argmax(index_predict)   # index

        prediction_main_list = list(training_set.class_indices.keys())
        prediction_id_main_list = list(training_set.class_indices.values())
        
        predicted_stage_name = prediction_main_list[predicted_value] # index of the current sleep stage in the prediction list
        id_stage_predict = stages_names_list.index(predicted_stage_name) + 1
        
        # print(f"Predicted value - {id_stage_predict}/{stage_id}")

        if id_stage_predict == stage_id:
            correct_predict_list[stage_id-1] += 1

        x_pred.append(start)
        y_pred.append(predicted_stage_name)
        x_pred.append(start + duration)
        y_pred.append(predicted_stage_name)

    print("======== Coordinates for true and predicted hypnogram have been received. ========")



    
    # Hypnogram plotting

    stages_names_for_plot = {
        "Sleep stage 4": 0,
        "Sleep stage 3": 1,
        "Sleep stage 2": 2,
        "Sleep stage 1": 3,
        "Sleep stage R": 4,        
        "Sleep stage W": 5
    }

    positions_of_labels = [0, 1, 2, 3, 4, 5]
    y_true_data_plot = [stages_names_for_plot[elem] for elem in y]
    y_pred_data_plot = [stages_names_for_plot[elem] for elem in y_pred]

    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 15

    fig = plt.figure()
    fig.set_figwidth(18)
    fig.set_figheight(10)

    plt.subplot(2, 1, 1)
    plt.plot(x, y_true_data_plot, '-b', linewidth='1', label="True hypnogram")
    plt.yticks(positions_of_labels, stages_names_for_plot)
    plt.xticks([])
    plt.xlabel('Time')
    plt.ylabel('Sleep stage')
    plt.title(f"Hypnograms for {img_folder_path[-7:]}")
    plt.legend()


    plt.subplot(2, 1, 2)
    plt.plot(x_pred, y_pred_data_plot, '-g', linewidth='1', label="Predicted hypnogram")
    plt.yticks(positions_of_labels, stages_names_for_plot)
    plt.xticks([])
    plt.xlabel('Time')
    plt.ylabel('Sleep stage')
    plt.legend()

    hypnogram_folder = img_folder_path + "/" + "hypnogram_info" + img_folder_path[-6:]
    if not os.path.isdir(hypnogram_folder):
        os.mkdir(hypnogram_folder)

    path_save_img = hypnogram_folder + f"/hypnogram_{img_folder_path[-6:]}"
    plt.savefig(path_save_img, bbox_inches='tight', pad_inches=0)

    plt.close(fig)

    # Creating confusion map

    
    for elem in y_true_data_plot:
        elem += 1
        
    for elem in y_pred_data_plot:
        elem += 1
    
    path_save_img = hypnogram_folder + f"/confusion_map_{img_folder_path[-6:]}"
    map = create_confusion_matrix(y_true_data_plot, y_pred_data_plot, path_save_img)


    # Percentage of predictions
    try:
        percent = round(sum(correct_predict_list)/sum(true_list) * 100, 2)
    except ZeroDivisionError:
        percent = 0

    info = f"All prediction results for {img_folder_path[-6:]}: {sum(correct_predict_list)}/{sum(true_list)} \
                    ({percent}%)\n"

    path_for_txt = hypnogram_folder + f"/predict_info_{img_folder_path[-6:]}.txt"
    with open(path_for_txt, "w") as file:
        file.write(info)

    print(info)

    for iteration in range(len(correct_predict_list)):
        try:
            percent = round(correct_predict_list[iteration]/true_list[iteration] * 100, 2)
        except ZeroDivisionError:
            if correct_predict_list[iteration] == true_list[iteration]:
                percent = 100
            else:
                percent = 0
        info = f"Prediction results for {stages_names_list[iteration]}: {correct_predict_list[iteration]}/{true_list[iteration]} \
                    ({percent}%)\n"
        print(info)

        with open(path_for_txt, "a") as file:
            file.write(info)
    
    print(f"======== Hypnogram {img_folder_path[-7:]} has been created. ========")

In [ ]:
def make_predict_create_hypns(cnn, dir_edf_files_predict, training_set):

    dir_predict_dataset = "dataset_predict"

    if not os.path.isdir(dir_predict_dataset):
        print("Prediction dataset hasn't been found. Impossible to make a prediction.")
        return

    if not cnn:
        cnn = tf.keras.models.load_model('models/exit_model.keras')
        
    print(f"Summary for cnn model '{model_path}':\n")
    cnn.summary()

    for folder in os.listdir(dir_predict_dataset):

        curr_path = dir_predict_dataset + "/" + folder

        psg_file = folder + "*"  # name of the true psg signal files (psg and hyp) in the dir_edf_files_predict
        possible_psg_files = fnmatch.filter(os.listdir(dir_edf_files_predict), psg_file)

        psg_file_path = ""
        hyp_file_path = ""
        for file in possible_psg_files:
            if file.split('-')[1][0] == "H":
                hyp_file_path = dir_edf_files_predict + "/" + file
            else:
                psg_file_path = dir_edf_files_predict + "/" + file

        data, annotations = file_keeper(psg_file_path, hyp_file_path)
        sfreq = data.info.get('sfreq')
        events_from_the_file, event_id_info, annotations_stage_id = crop_set_annotations(data, annotations)

        tmax = 30.0 - 1.0 / data.info["sfreq"]
        epochs_from_the_file = mne.Epochs(
            raw=data,
            events=events_from_the_file,
            event_id=event_id_info,
            tmin=0.0,
            tmax=tmax,
            baseline=None,
            verbose=False
        )
        events = epochs_from_the_file.get_data(picks=[0])
        times_events = epochs_from_the_file.events

        print(f"Current path: {curr_path}")
        create_hypnograms(events, times_events, sfreq, curr_path, cnn, annotations_stage_id, training_set)

    print("All hypnograms have been created.")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

def create_confusion_matrix(true_array, prediction_array, path_save_img=''):
    labels_stages = [1, 2, 3, 4, 5, 6]
    result = confusion_matrix(true_array, prediction_array, labels=labels_stages)
    print(f"\nConfusion matrix:\n{result}\n")

    table = pd.DataFrame(result, range(len(labels_stages)), range(len(labels_stages)))
    fig = plt.figure()
    map = sn.heatmap(table, annot=True, fmt='g')

    if path_save_img:
        plt.savefig(path_save_img)
    else:
        plt.show()
    plt.close(fig)

# EXIT CODE

## CREATING DATASETS

In [ ]:
create_directories()

print("\n\n======== CREATING TRAINING SIGNALS ========\n\n")
dir = "edf_files/sleep-cassette"
optimal_number_images, max_number_images = find_min_num_of_file(dir)
dataset_create(dir, optimal_number_images, "train")

print("\n\n======== CREATING TEST SIGNALS ========\n\n")
dir = "edf_files/x_test_y_test"
optimal_number_images, max_number_images = find_min_num_of_file(dir)
dataset_create(dir, optimal_number_images, "test")


print("\n\n======== CREATING VALIDATION SIGNALS ========\n\n")
dir = "edf_files/x_valid_y_valid"
optimal_number_images, max_number_images = find_min_num_of_file(dir)
dataset_create(dir, optimal_number_images, "valid")

## CREATING PREDICTING DATASET

In [ ]:
files_directory = "edf_files/x_test_y_test"
create_predict_dataset(files_directory)

## Training CNN model

In [ ]:
check_for_hid_files()
training_set, validation_set, test_set = preprocessing_func()
cnn = build_cnn()
cnn_train_model(cnn, training_set, validation_set)

In [ ]:
loss_value, acc_value = cnn.evaluate(test_set)

print(f"Loss value: {loss_value}")
print(f"Accuracy value: {acc_value}")

## PREDICTING

In [ ]:
dir_edf_files_predict = "edf_files/x_test_y_test"
model_path = 'models/exit_model.keras'
make_predict_create_hypns(_, dir_edf_files_predict, training_set)